## Building a BATTLE Eval

In [334]:
new = True
initial_message = [{"role": "user", "content": ""}]
battles_generation = "dense"      # methods: dense, only-new
roles = "system-user"             # methods: system-user, system, user

#!cd evals
#!git lfs fetch --all
#!git lfs pull

In [335]:
import os
from openai import OpenAI
import pandas as pd
import yaml
#import datetime
#from dotenv import load_dotenv

#load_dotenv()  # Load environment variables from .env file

#api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

# Install Evals if you haven't already
# %pip install -e ../.
# pip install --upgrade openai
# %pip install pandas


In [336]:
# Paths. Assuming this notebook is in examples/

evals_path = os.path.join(os.getcwd(), "..", "evals")

registry_path = os.path.join(evals_path, "registry", "evals", "battles.yaml")

data_path = os.path.join(evals_path, "registry", "data", "battles")
os.makedirs(data_path, exist_ok=True)
data_path = os.path.join(data_path, "samples.jsonl")

json_logs_path = os.path.join(os.getcwd(), "logs")
os.makedirs(json_logs_path, exist_ok=True)
json_logs_path = os.path.join(json_logs_path, "logs")

df_path = os.path.join(evals_path, "evallogs", "df")
os.makedirs(df_path, exist_ok=True)
dataset_path = os.path.join(df_path, "dataset")

In [337]:
# Registry yaml

registry_yaml = {}

registry_yaml["battles"] = {
    "id": "battles.test.v1",
    "metrics": ["accuracy"]
}
registry_yaml["battles.test.v1"] = {
    "class": "evals.elsuite.modelgraded.classify:ModelBasedClassify",
    "args": {
        "samples_jsonl": "battles/samples.jsonl",
        "eval_type": "cot_classify",
        "modelgraded_spec": "battle"
    }
}
with open(os.path.join(registry_path), "w") as f:
    yaml.dump(registry_yaml, f)

In [338]:
# Initial data
if new:
    dataset = ()
    dataset = pd.DataFrame(dataset, columns=['Instruction1', 'Response1', 'Instruction2', 'Response2', 'Sampled',
       'Choice', 'Data'])
    best_message = initial_message
    best_response = client.chat.completions.create(
        messages = initial_message,
        model = "gpt-3.5-turbo",
        ).choices[0].message.content
else:
    with open(dataset_path, "r") as f:
        dataset = pd.read_json(f, lines=True)
    if dataset.iloc[-1]["Choice"] == "No":
        best_content = dataset.iloc[-1]["Instruction2"]
        best_response = dataset.iloc[-1]["Response2"]
    else:
        best_content = dataset.iloc[-1]["Instruction1"]
        best_response = dataset.iloc[-1]["Response1"]
    best_message = [{"role": "user", "content": best_content}]
candidate_messages = [best_message]
next_candidate_messages = []
generation_distance = 0


In [339]:
def battle(best_message, best_response, candidate_message):

    dataset = [{"input1": candidate_message, "input2": best_message, "completion2":best_response}]

    df = pd.DataFrame(dataset)
    df.to_json(data_path, orient="records", lines=True)

    !oaieval gpt-3.5-turbo battles --record_path logs/logs
    
    with open(json_logs_path, "r") as f:
        df = pd.read_json(f, lines=True)

    #current_time = datetime.datetime.now()
    #formatted_time = current_time.strftime("%Y%m%d_%H%M%S")
    #df.to_json(os.path.join(df_path, formatted_time), lines=True, orient="records")

    instruction1 = candidate_message[0]["content"]
    instruction2 = best_message[0]["content"]

    battle_prompt_content = df["data"].iloc[-2]["prompt"][0]["content"]
    response1 = battle_prompt_content.split("\n[Response 1]\n",)[1].split("\n\n[Instruction 2]\n")[0]
    response1 = response1.replace("\\'", "'").replace("\\n", "\n")
    response2 = battle_prompt_content.split("\n[Response 2]\n",)[1].split("\n\n\nIs the first response better than the second?")[0]
    response2 = response2.replace("\\'", "'").replace("\\n", "\n")
    print(f"Response1: {response1}")
    #print(f"response2: {response2}")

    sampled = df["data"].iloc[-2]["sampled"][0]

    choice = df["data"].iloc[-1]["choice"]

    data = {'Instruction1': instruction1, 'Response1': response1, 'Instruction2': instruction2, 'Response2': response2, 'Sampled': sampled, 'Choice': choice, 'Data': {}}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = df.to_dict()
    return data

In [340]:
def switch_system_and_user(message):
    new_message = []
    if message[0]["role"] == "system":
        new_message.append([{"role": "user", "content": message[0]["content"]}])
    else:
        new_message.append([{"role": "system", "content": message[0]["content"]}])
    return new_message

In [341]:
def find_parents(content, dataset):
    print (f"finding parents {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Response1"] == content]
    parents = dataset["Instruction1"].unique()
    return parents

In [342]:
def find_children(content, dataset):
    print (f"finding children {content}")
    dataset = pd.DataFrame(dataset)
    dataset = dataset[dataset["Instruction1"] == content]
    children = dataset["Response1"].unique()
    return children

In [343]:
def list_candidate_messages(dataset, best_message, generation_distance):
    best_content = best_message[0]["content"]
    last_up = [best_content]
    last_down = [best_content]
    list_of_contents = [best_content]

    next_up = []
    next_down = []
    for i in range(generation_distance):
        print (f"starting level {i}, generation distance {generation_distance}")
        for content in last_up:
            #print(f"last_up: {last_up}, now {content}")
            next_up.extend(find_parents(content, dataset))
        for content in last_down:
            #print(f"last_up: {last_down}, now {content}")
            next_down.extend(find_children(content, dataset))
        print (f"behind for loops")
        list_of_contents.extend(next_up)
        list_of_contents.extend(next_down)
        last_up = next_up.copy()
        last_down = next_down.copy()
        print(f"endind level {i}")
        

    list_of_contents = pd.array(list_of_contents).unique().tolist()
    
    #best_message_index = list_of_contents.index(best_message[0][content])
    #start = max(best_message_index - generation_distance, 0)
    #stop = min(len(list_of_contents))
    #stop = len(list_of_contents) - 1

    messages = []
    for content in list_of_contents:
        if roles == "system-user" or "user":
            messages.append([{"role":"user","content":content}])
        if roles == "system-user" or "system":
            messages.append([{"role":"system","content":content}])
    return messages

In [374]:
for candidate_message in candidate_messages:
    data = battle(best_message, best_response, candidate_message)
    dataset = pd.concat([dataset, data],ignore_index=True)
    dataset.to_json(os.path.join(df_path, "dataset"), lines=True, orient="records")
    if data["Choice"].iloc[0] == "Yes":
        best_message = candidate_message
        best_response = data["Response1"].iloc[0]
        print(f"New best message:{best_message}")
        if battles_generation == "dense":
            generation_distance = 0
    print(f"Instruction1: {candidate_message[0]["content"]}")

    if battles_generation == "only-new":
        new_message = [{"role": candidate_message[0]["role"], "content":data["Response1"].iloc[0]}]
        if new_message not in next_candidate_messages:
            next_candidate_messages.append(new_message)
            if roles == "system-user":
                next_candidate_messages.append(switch_system_and_user(new_message))

if battles_generation == "only-new":
    candidate_messages = next_candidate_messages
    next_candidate_messages = []
if battles_generation == "dense":
    #print("going to list candidate messages")
    candidate_messages = list_candidate_messages(dataset, best_message, generation_distance)
    generation_distance += 1
    #print("candidate messages listed")
print(f"all done, generation distance {generation_distance}")

[2024-08-09 09:58:54,140] [registry.py:271] Loading registry from /Users/janvotava/Desktop/evals/evals/registry/evals
[2024-08-09 09:58:54,947] [registry.py:271] Loading registry from /Users/janvotava/.evals/evals
[2024-08-09 09:58:54,952] [oaieval.py:215] Run started: 2408090758545O4GDFMN
[2024-08-09 09:58:54,954] [registry.py:271] Loading registry from /Users/janvotava/Desktop/evals/evals/registry/modelgraded
[2024-08-09 09:58:54,979] [registry.py:271] Loading registry from /Users/janvotava/.evals/modelgraded
[2024-08-09 09:58:54,980] [data.py:94] Fetching /Users/janvotava/Desktop/evals/evals/registry/data/battles/samples.jsonl
[2024-08-09 09:58:54,980] [eval.py:36] Evaluating 1 samples
[2024-08-09 09:58:55,007] [eval.py:144] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.21s/it]
[2024-08-09 09:58:57,223] [oaieval.py:275] Found 2/2 sampling events with usage data
[2024-08-09 09:58:57,223] [oaieval.py:283] Token usage

In [372]:
dataset.iloc[-10:]

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
17,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses are very similar in content ...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
18,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to help with any questions you...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses acknowledge the purpose of b...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
19,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses are very similar in content ...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
20,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to help with any questions you...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses acknowledge the purpose of h...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
21,Hello! How can I assist you today?,Hello! I'm here to help with any questions or ...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses start with a friendly greeti...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
22,Hello! How can I assist you today?,Hello! How can I assist you today?,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses start with a friendly greeti...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
23,Hello! I'm here to assist you with any questio...,I have a question about my recent order. Can y...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Response 1 directly addresses the specific ...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
24,Hello! I'm here to assist you with any questio...,Hello! How can I assist you today?,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. The first response in Instruction 1 is more...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
25,Hello! I'm here to help with any questions you...,I can help with a variety of topics such as ge...,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Response 1 provides a more detailed list of...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...
26,Hello! I'm here to help with any questions you...,Hello! How can I assist you today?,Hello! I'm just here to chat and answer any qu...,Hello! I'm here to assist you with any questio...,1. Both responses start with a friendly greeti...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...


In [346]:
dataset#.iloc[166:]

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...


In [347]:
from IPython.display import display_markdown

In [348]:
display_markdown(best_message[0]["content"], raw=True)

In [349]:
display_markdown(best_response, raw=True)

Hello! How can I assist you today?

In [369]:
for entry in dataset[dataset["Choice"]=="Yes"].iloc[-1]:
    display_markdown(entry, raw=True)

Hello! I'm just here to chat and answer any questions you may have. How can I help you today?

Hello! I'm here to assist you with any questions or concerns you may have. How can I help you today?

Hello! How can I assist you today?

Hello! I'm just here to chat and answer any questions you may have. How can I help you today?

1. Response 1 directly addresses the user's potential questions or concerns, showing a proactive approach to assisting them.
2. Response 2 does not directly address the user's potential questions or concerns, instead focusing on chatting and answering questions.

Based on the direct approach and focus on addressing questions or concerns in Response 1, it is better than Response 2.

Yes

Yes

Yes

ValueError: NaTType does not support strftime

In [351]:
data

,Instruction1,Response1,Instruction2,Response2,Sampled,Choice,Data
0,,Hello! How can I assist you today?,,Hello! How can I assist you today?,1. Both responses are identical in content and...,No,{'spec': {0: {'completion_fns': ['gpt-3.5-turb...


In [373]:
candidate_messages

[[{'role': 'user',
   'content': "Hello! I'm just here to chat and answer any questions you may have. How can I help you today?"}],
 [{'role': 'system',
   'content': "Hello! I'm just here to chat and answer any questions you may have. How can I help you today?"}],
 [{'role': 'user', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'system', 'content': 'Hello! How can I assist you today?'}],
 [{'role': 'user',
   'content': "Hello! I'm here to assist you with any questions or concerns you may have. How can I help you today?"}],
 [{'role': 'system',
   'content': "Hello! I'm here to assist you with any questions or concerns you may have. How can I help you today?"}],
 [{'role': 'user',
   'content': "Hello! I'm here to help with any questions you have. What would you like to know?"}],
 [{'role': 'system',
   'content': "Hello! I'm here to help with any questions you have. What would you like to know?"}],
 [{'role': 'user', 'content': ''}],
 [{'role': 'system', 'content': ''